In [39]:
import input, text_processing
import pandas as pd
import numpy as np
# Imports used in the tweet text processing
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_selection import chi2
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

# Imports used to build the autoencoder
import keras
from keras.datasets import mnist
from keras.models import Model, Sequential
from keras.layers import Input, Dense, Flatten, Reshape

#% load_ext autoreload
#% autoreload 2
% matplotlib inline



Using TensorFlow backend.


In [19]:
df = input.read_sample_dataset()
df.head()

,id_str,text,polarity,subjectivity,user_name,user_location,user_followers,created,retweet_count
0,1049367463295508480,Vamos eleger a educação para combater a violên...,0.0,0,Reinald71694969,"Xique-Xique, Brasil",4,2018-10-08 18:34:34.000000,0.0
1,1049367465778712576,#Bolsonaro2018 1⃣7⃣😎😎 https://t.co/jasNso3sBt,0.0,0,Constan_Daniel,NaN,147,2018-10-08 18:34:34.000000,0.0
2,1049367475190747137,RT @Horacio_Giusto: Lo sucedido ayer con #Bols...,0.0,0,herkow,Antártida,451,2018-10-08 18:34:37.000000,0.0
3,1049367475991863302,@g1 Então vai ser #BolsonaroPresidente17 @jair...,0.0,0,ArbexSilva,São Paulo,172,2018-10-08 18:34:37.000000,0.0
4,1049367482732097536,Bienvenidos a Charly y la fabrica de chocolate...,0.0,0,Pgascoin,NaN,155,2018-10-08 18:34:38.000000,0.0


In [16]:
cols = ['text', 'polarity', 'user_followers']
df = df[cols]
df = df[pd.notnull(df.text)]
df.head()

,text,polarity,user_followers
0,Vamos eleger a educação para combater a violên...,0.0,4
1,#Bolsonaro2018 1⃣7⃣😎😎 https://t.co/jasNso3sBt,0.0,147
2,RT @Horacio_Giusto: Lo sucedido ayer con #Bols...,0.0,451
3,@g1 Então vai ser #BolsonaroPresidente17 @jair...,0.0,172
4,Bienvenidos a Charly y la fabrica de chocolate...,0.0,155


In [19]:
tfidf = TfidfVectorizer(sublinear_tf=True, 
                        min_df=10, norm='l2', 
                        encoding='latin-1', 
                        ngram_range=(1, 2), 
                        stop_words=text_processing.pt_br_stop_words)
features = tfidf.fit_transform(df.text).toarray()
features.shape
tfidf.get_feature_names()

['091pmfw7zv',
 '10',
 '10 2018',
 '100',
 '100 milioni',
 '100 millones',
 '11',
 '12',
 '12 puñaladas',
 '13',
 '17',
 '17neles',
 '17neles bolsonaro17',
 '18',
 '1m',
 '1st',
 '1st round',
 '1º',
 '1º siga',
 '20',
 '2011',
 '2011 59',
 '2018',
 '28',
 '28 10',
 '29',
 '2da',
 '2smq7cwnow',
 '2º',
 '2º favorite',
 '2º turno',
 '30',
 '32',
 '37',
 '37 acabou',
 '3dn61popwg',
 '3º',
 '3º abra',
 '40',
 '40 milhões',
 '46',
 '47',
 '49',
 '4ovmeylroc',
 '4sasd7vezt',
 '4º',
 '52',
 '52 câmara',
 '59',
 '59 haddad',
 '59 universidades',
 '5sgkvrwvtj',
 '7mmoibxgoy',
 '85',
 '85 senado',
 'about',
 'about jairbolsonaro',
 'about the',
 'abra',
 'abra tweet',
 'abstenções',
 'abstenções 29',
 'acaba',
 'acaba pra',
 'acabamos',
 'acabamos co',
 'acabar',
 'acabei',
 'acabou',
 'acabou declarar',
 'acabou pt',
 'aceitar',
 'acha',
 'acha validação',
 'acham',
 'acho',
 'acima',
 'acima todos',
 'acima tudo',
 'acorda',
 'acordei',
 'acordei igualzin',
 'acredito',
 'activaron',
 'activaro

In [37]:
text_input = df.text
count_vect = CountVectorizer(min_df=10, encoding='latin-1', ngram_range=(1, 2), stop_words=text_processing.pt_br_stop_words)

text_counts = count_vect.fit_transform(text_input)
tfidf_transformer = TfidfTransformer(sublinear_tf=True, norm='l2')
text_freq_input = tfidf_transformer.fit_transform(text_counts)

input_array = text_freq_input.toarray()
input_array.shape

(9994, 2615)

In [47]:
input_neurons = input_array.shape[1]
encoding_neurons = 32

compression_factor = float(input_neurons) / encoding_neurons
print("Compression factor: %s" % compression_factor)

autoencoder = Sequential()

# Encoder Layers
autoencoder.add(Dense(4 * encoding_neurons, input_shape=(input_neurons,), activation='relu'))
autoencoder.add(Dense(2 * encoding_neurons, activation='relu'))
autoencoder.add(Dense(encoding_neurons, activation='relu'))

# Decoder Layers
autoencoder.add(Dense(2 * encoding_neurons, activation='relu'))
autoencoder.add(Dense(4 * encoding_neurons, activation='relu'))
autoencoder.add(Dense(input_neurons, activation='sigmoid'))


autoencoder.summary()

Compression factor: 81.71875
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_15 (Dense)             (None, 128)               334848    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
_________________________________________________________________
dense_18 (Dense)             (None, 64)                2112      
_________________________________________________________________
dense_19 (Dense)             (None, 128)               8320      
_________________________________________________________________
dense_20 (Dense)             (None, 2615)              337335    
Total params: 692,951
Trainable params: 692,951
Non-trainable params: 0
_________________________________________

In [49]:
# save the encoding layer to use later as input to the clustering algorithm
input_layer = Input(shape=(input_neurons,))
encoder_layer1 = autoencoder.layers[0]
encoder_layer2 = autoencoder.layers[1]
encoder_layer3 = autoencoder.layers[2]
encoder = Model(input_layer, encoder_layer3(encoder_layer2(encoder_layer1(input_layer))))

encoder.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 2615)              0         
_________________________________________________________________
dense_15 (Dense)             (None, 128)               334848    
_________________________________________________________________
dense_16 (Dense)             (None, 64)                8256      
_________________________________________________________________
dense_17 (Dense)             (None, 32)                2080      
Total params: 345,184
Trainable params: 345,184
Non-trainable params: 0
_________________________________________________________________


In [51]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')
autoencoder.fit(input_array, input_array,
                epochs=50,
                batch_size=256,
                shuffle=True,
                validation_data=(input_array, input_array))


Train on 9994 samples, validate on 9994 samples
Epoch 1/50
9994/9994 [==============================] - 5s 509us/step - loss: 0.0049 - val_loss: 0.0048
Epoch 2/50
9994/9994 [==============================] - 5s 453us/step - loss: 0.0048 - val_loss: 0.0047
Epoch 3/50
9994/9994 [==============================] - 4s 446us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 4/50
9994/9994 [==============================] - 4s 444us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 5/50
9994/9994 [==============================] - 5s 451us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 6/50
9994/9994 [==============================] - 4s 418us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 7/50
9994/9994 [==============================] - 4s 387us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 8/50
9994/9994 [==============================] - 4s 388us/step - loss: 0.0047 - val_loss: 0.0047
Epoch 9/50
9994/9994 [==============================] - 4s 412us/step - loss: 0.0047 - val_loss: 0.0046
Epoch 10/50
9994